In [5]:
from goprocam import GoProCamera, constants
from threading import Thread
import cv2
import socket
import math
import numpy as np
from time import time
WRITE = False
#setup the tracker, read more https://ehsangazar.com/object-tracking-with-opencv-fd18ccdd7369
tracker = cv2.TrackerKCF_create()
#stream the video from webcam
cap=cv2.VideoCapture(1)
ret,frame = cap.read()
center = []
frame_index = 0    
#select a bounding box to do object tracking
bbox = cv2.selectROI(frame, False,False)
# Initialize tracker with first frame and bounding box
ok = tracker.init(frame, bbox)
while True:
    ret, frame = cap.read()
    cv2.imshow("webcam", frame)
   # hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
   # mask = cv2.inRange(hsv,lower_bound,upper_bound)
   # res = cv2.bitwise_and(frame,frame,mask=mask)
   # gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
   # cv2.imshow("res",res)
   # gray = cv2.medianBlur(gray,21)
   # kernel = np.ones((5, 5), np.uint8)
   # mask = cv2.erode(gray, kernel)
# Update tracker
    ok, bbox = tracker.update(frame)
    if ok:
         # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
        center.append([frame_index, int(bbox[0] + bbox[2]/2), int(bbox[1] + bbox[3]/2)])
        #draw center
        cv2.circle(frame,(int(bbox[0] + bbox[2]/2),int(bbox[1] + bbox[3]/2)),2,(0,0,255),3)
    else :
            # Tracking failure
            cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)
    cv2.imshow("tracking",frame)
    np.savetxt("web_cam.csv", center, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
import cv2
import socket
import math
import imutils
import numpy as np
from time import time
from matplotlib import pyplot as plt
WRITE = False
center1 = [] #has to differ than the previous cut
frame_index = 0
#define lower and upper boundries for black color
lower_bound = np.array([0,0,0])
upper_bound = np.array([96,96,96])#before it's 96,96,96
#stream the video from webcam
cap=cv2.VideoCapture(0)
frames_per_second = 20.0
out = None
#create a black image and an array to contain three points
img = np.zeros((512,512,3), np.uint8)
points = []
while True:
    ret,frame = cap.read()
    #get the frame size
    if ret:
        frame_dimension = frame.shape
        frame_height = frame_dimension[0]
        frame_width = frame_dimension[1]
        #define the codec and create a video_writer object
        if out is None:
                fourcc = cv2.VideoWriter_fourcc(*'mp4v') # Be sure to use the lower case
#                 out  = cv2.VideoWriter('center_points_{0}.mp4',fingerVid9)
                out = cv2.VideoWriter('test_web_cam.mp4',
                             fourcc, frames_per_second, (frame_width, frame_height), True)
    
    cv2.imshow("webcam",frame)
    #blur the picture and transform it to hsv
    blurred = cv2.GaussianBlur(frame,(11,11),0)
    hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)
    #construct a mask of black
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    mask = cv2.erode(mask, None, iterations = 2)
    mask = cv2.dilate(mask, None, iterations = 2)
    #find contours in the mask
    cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    cnts = imutils.grab_contours(cnts)
    for cn in cnts:
        c = max(cn, key=cv2.contourArea)
        ((x,y),radius) = cv2.minEnclosingCircle(cn)
        if radius > 15:#this may change due to the size of mark
            cv2.circle(frame,(int(x),int(y)),int(radius),(255,255,255),-1)
            #fill up a point array
            points.append((int(x),int(y)))
            #fill up the array for csv file
            center1.append([frame_index, int(x), int(y)])
            #print("frame is",frame_index,"x is", int(x), "y is", int(y))
    frame_index = frame_index + 1
    for size in range(len(points)-1):
        cv2.line(img,points[size],points[size+1],(255,0,0),5)
         #cv2.drawContours(frame,[c],-1,(255,255,255),2)
    points = []
    out.write(frame)
    cv2.imshow("graph",img)
    #renew the image
    img = np.zeros((512,512,3), np.uint8)
    cv2.imshow("detect",frame)
    #Write modified video to output csv file
    np.savetxt("web_cam1.csv", center1, delimiter=",", fmt='%d', header= "Frame Number, X, Y", comments = '')#the csv file name and array should be matched
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()

points are [(407, 426), (336, 392), (250, 365), (167, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (167, 331)
points are (167, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (167, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (167, 331)
points are (167, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (167, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (167, 331)
points are (167, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (167, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (167, 331)
points are (167, 331) 

points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 427), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 427) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 427), (336, 392), (250, 365), (166, 331), (85, 307)]
points are (407, 427) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 307)
points are [(407, 427), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 427) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 427), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 427) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (167, 331), (

points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 307)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 307)
points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) (85, 306)
points are [(407, 426), (336, 392), (250, 365), (166, 331), (85, 306)]
points are (407, 426) (336, 392)
points are (336, 392) (250, 365)
points are (250, 365) (166, 331)
points are (166, 331) 

points are (107, 265) (191, 212)
points are (191, 212) (252, 113)
points are [(107, 258), (187, 196)]
points are (107, 258) (187, 196)
points are [(108, 253), (183, 184), (18, 71), (214, 68)]
points are (108, 253) (183, 184)
points are (183, 184) (18, 71)
points are (18, 71) (214, 68)
points are [(108, 252), (182, 180), (10, 91), (203, 58)]
points are (108, 252) (182, 180)
points are (182, 180) (10, 91)
points are (10, 91) (203, 58)
points are [(107, 252), (182, 181), (12, 107), (199, 56), (83, 4)]
points are (107, 252) (182, 181)
points are (182, 181) (12, 107)
points are (12, 107) (199, 56)
points are (199, 56) (83, 4)
points are [(108, 252), (182, 181), (14, 118), (198, 55), (77, 9)]
points are (108, 252) (182, 181)
points are (182, 181) (14, 118)
points are (14, 118) (198, 55)
points are (198, 55) (77, 9)
points are [(108, 249), (180, 176), (13, 130), (190, 49), (65, 14)]
points are (108, 249) (180, 176)
points are (180, 176) (13, 130)
points are (13, 130) (190, 49)
points are (190

points are (108, 244) (170, 160)
points are (170, 160) (15, 37)
points are (15, 37) (145, 30)
points are [(108, 248), (174, 168), (160, 37), (31, 28)]
points are (108, 248) (174, 168)
points are (174, 168) (160, 37)
points are (160, 37) (31, 28)
points are [(108, 252), (179, 177), (176, 46), (53, 16)]
points are (108, 252) (179, 177)
points are (179, 177) (176, 46)
points are (176, 46) (53, 16)
points are [(107, 257), (183, 186), (193, 58)]
points are (107, 257) (183, 186)
points are (183, 186) (193, 58)
points are [(108, 261), (186, 196), (211, 72)]
points are (108, 261) (186, 196)
points are (186, 196) (211, 72)
points are [(106, 265), (189, 206), (228, 89)]
points are (106, 265) (189, 206)
points are (189, 206) (228, 89)
points are [(106, 270), (191, 216), (243, 108)]
points are (106, 270) (191, 216)
points are (191, 216) (243, 108)
points are [(105, 274), (193, 227), (256, 129)]
points are (105, 274) (193, 227)
points are (193, 227) (256, 129)
points are [(103, 279), (194, 241), (2

...

points are (434, 377) (353, 359)
points are (353, 359) (260, 348)
points are (260, 348) (171, 324)
points are (171, 324) (87, 304)
points are [(434, 376), (353, 359), (261, 347), (171, 324), (87, 304)]
points are (434, 376) (353, 359)
points are (353, 359) (261, 347)
points are (261, 347) (171, 324)
points are (171, 324) (87, 304)
points are [(434, 376), (353, 359), (261, 347), (171, 324), (87, 304)]
points are (434, 376) (353, 359)
points are (353, 359) (261, 347)
points are (261, 347) (171, 324)
points are (171, 324) (87, 304)
points are [(434, 377), (352, 360), (260, 348), (171, 324), (87, 304)]
points are (434, 377) (352, 360)
points are (352, 360) (260, 348)
points are (260, 348) (171, 324)
points are (171, 324) (87, 304)
points are [(433, 378), (352, 360), (260, 348), (171, 324), (87, 304)]
points are (433, 378) (352, 360)
points are (352, 360) (260, 348)
points are (260, 348) (171, 324)
points are (171, 324) (87, 304)
points are [(433, 378), (352, 360), (260, 348), (171, 324), (

points are (421, 402) (345, 374)
points are (345, 374) (256, 355)
points are (256, 355) (169, 327)
points are (169, 327) (86, 305)
points are [(418, 408), (343, 379), (255, 358), (168, 328), (86, 305)]
points are (418, 408) (343, 379)
points are (343, 379) (255, 358)
points are (255, 358) (168, 328)
points are (168, 328) (86, 305)
points are [(411, 420), (338, 388), (251, 363), (166, 331), (85, 306)]
points are (411, 420) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(413, 419), (339, 387), (252, 363), (166, 331), (85, 306)]
points are (413, 419) (339, 387)
points are (339, 387) (252, 363)
points are (252, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(415, 413), (341, 383), (253, 360), (167, 330), (86, 306)]
points are (415, 413) (341, 383)
points are (341, 383) (253, 360)
points are (253, 360) (167, 330)
points are (167, 330) (86, 306)
points are [(418, 408), (343, 379), (254, 358), (168, 329), (

points are (413, 417) (340, 385)
points are (340, 385) (252, 361)
points are (252, 361) (167, 330)
points are (167, 330) (85, 306)
points are [(413, 417), (340, 385), (253, 361), (167, 330), (86, 306)]
points are (413, 417) (340, 385)
points are (340, 385) (253, 361)
points are (253, 361) (167, 330)
points are (167, 330) (86, 306)
points are [(413, 417), (340, 385), (252, 361), (167, 330), (85, 306)]
points are (413, 417) (340, 385)
points are (340, 385) (252, 361)
points are (252, 361) (167, 330)
points are (167, 330) (85, 306)
points are [(413, 418), (340, 385), (252, 362), (167, 331), (85, 306)]
points are (413, 418) (340, 385)
points are (340, 385) (252, 362)
points are (252, 362) (167, 331)
points are (167, 331) (85, 306)
points are [(413, 418), (340, 386), (252, 362), (167, 330), (85, 306)]
points are (413, 418) (340, 386)
points are (340, 386) (252, 362)
points are (252, 362) (167, 330)
points are (167, 330) (85, 306)
points are [(413, 418), (340, 386), (252, 362), (166, 331), (

points are (412, 419) (339, 386)
points are (339, 386) (252, 362)
points are (252, 362) (166, 331)
points are (166, 331) (85, 306)
points are [(412, 419), (339, 387), (252, 362), (167, 331), (85, 306)]
points are (412, 419) (339, 387)
points are (339, 387) (252, 362)
points are (252, 362) (167, 331)
points are (167, 331) (85, 306)
points are [(412, 419), (339, 387), (251, 362), (166, 331), (85, 306)]
points are (412, 419) (339, 387)
points are (339, 387) (251, 362)
points are (251, 362) (166, 331)
points are (166, 331) (85, 306)
points are [(412, 419), (339, 387), (252, 362), (166, 331), (85, 306)]
points are (412, 419) (339, 387)
points are (339, 387) (252, 362)
points are (252, 362) (166, 331)
points are (166, 331) (85, 306)
points are [(412, 419), (339, 387), (252, 362), (166, 331), (86, 306)]
points are (412, 419) (339, 387)
points are (339, 387) (252, 362)
points are (252, 362) (166, 331)
points are (166, 331) (86, 306)
points are [(412, 420), (339, 387), (252, 362), (166, 331), (

points are (411, 421) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(411, 421), (338, 388), (251, 363), (166, 331), (85, 307)]
points are (411, 421) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 307)
points are [(411, 421), (338, 388), (251, 363), (166, 331), (85, 306)]
points are (411, 421) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(411, 421), (338, 388), (251, 363), (166, 331), (85, 306)]
points are (411, 421) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(411, 421), (338, 388), (251, 363), (166, 331), (85, 306)]
points are (411, 421) (338, 388)
points are (338, 388) (251, 363)
points are (251, 363) (166, 331)
points are (166, 331) (85, 306)
points are [(411, 421), (338, 388), (251, 363), (166, 331), (